# call edsites in all regions that are
1. in a gene
2. not within 4bp of a splice junction
3. not in homopolymer run of 5+ bp
4. not in simple repeats (I use the excluderanges(https://dozmorovlab.github.io/excluderanges/articles/excluderanges.html) list to achive something similar)

In [1]:
import pandas as pd
import pyranges as pr
import re
from tqdm.notebook import tqdm
import numpy as np

/home/klawren/.pixi/envs/python/lib/python3.12/site-packages/sorted_nearest/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


make the merged gene regions with `/home/klawren/oak/gtex/scripts/analysis/create_hc_gene_intervals.sh`

here I make the regions to remove

In [ ]:
# load in gencode 47 gtf
gencode_v47_path = "/home/klawren/oak/gtex/data/realign_references/gencode.v47.annotation.gtf"
gencode_v47=pr.read_gtf(gencode_v47_path)
gencode_v47_df = gencode_v47.as_df()

In [ ]:
# pull out all exons, the creat cresion of +- 4 around the junctions between exons
exons = gencode_v47_df[gencode_v47_df['Feature'] =='exon']
max_exon_number = exons.groupby('gene_id').agg({'exon_number':'max'})
exons = pd.merge(exons, max_exon_number, on='gene_id', how='left', suffixes=['', '_max'])


splice_sites_exon_starts = exons[['Chromosome' ,'Start', 'exon_number', 'exon_number_max']]
splice_sites_exon_starts['End'] = splice_sites_exon_starts['Start'] + 4
splice_sites_exon_starts['Start'] = splice_sites_exon_starts['Start'] - 4

splice_sites_exon_ends = exons[['Chromosome' ,'End', 'exon_number', 'exon_number_max']]
splice_sites_exon_ends['Start'] = splice_sites_exon_ends['End'] - 4
splice_sites_exon_ends['End'] = splice_sites_exon_ends['End'] + 4

# dont pad around the start of the first exon or end of the last exon
splice_sites_exon_starts = splice_sites_exon_starts[splice_sites_exon_starts['exon_number'] != 1]
splice_sites_exon_ends = splice_sites_exon_ends[splice_sites_exon_ends['exon_number'] != splice_sites_exon_ends['exon_number_max']]

splice_site_bed = pd.concat([splice_sites_exon_ends[['Chromosome', 'Start', 'End']], splice_sites_exon_starts[['Chromosome', 'Start', 'End']]])
splice_site_bed.to_csv('/home/klawren/oak/gtex/data/edsite_references/gencode.v47.splice_site.bed', sep='\t', index=None, header=None)

/local/scratch/klawren/slrmtmp.48466672/ipykernel_3940/2723239820.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  splice_sites_exon_starts['End'] = splice_sites_exon_starts['Start'] + 4
/local/scratch/klawren/slrmtmp.48466672/ipykernel_3940/2723239820.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  splice_sites_exon_starts['Start'] = splice_sites_exon_starts['Start'] - 4
/local/scratch/klawren/slrmtmp.48466672/ipykernel_3940/2723239820.py:11: SettingWithCopyWarning: 
A value is trying to be set on a

pull out any homopolymer runs in the gene regions

In [ ]:
gene_intervals = pr.read_bed("/home/klawren/oak/gtex/data/edsite_references/gencode.v47.merged.bed")
gene_intervals_df = gene_intervals.as_df()

# pull the sequnece for the intervals
gene_interval_seq = pr.get_sequence(gene_intervals, "/home/klawren/oak/gtex/data/edsite_references/Homo_sapiens_assembly38_noALT_noHLA_noDecoy.fasta")

In [ ]:
# find coordinates of homopolymer runs

def find_homopolymer(sequence):
    pattern = r'([ACTG])\1{4,}' # run of 5 or more
    matches = re.finditer(pattern, sequence)
    
    indices = []
    for match in matches:
        start_index = match.start()
        end_index = match.end() - 1 
        indices.append((start_index, end_index))
    return np.asarray(indices)

# loop through each gene region
homopolymer_runs = []
for idx in tqdm(range(len(gene_intervals_df))):
    gene_interval = gene_intervals_df.iloc[idx]
    run_idxs = find_homopolymer(gene_interval_seq[idx])
    if len(run_idxs) > 0:
        run_idxs += gene_interval['Start']
        run_idxs = pd.DataFrame({'Start':run_idxs[:,0], 'End':run_idxs[:,1]})
        run_idxs['Chromosome'] = gene_interval['Chromosome']
        homopolymer_runs.append(run_idxs)

    
# write out
pd.concat(homopolymer_runs)[['Chromosome', 'Start', 'End']].to_csv('/home/klawren/oak/gtex/data/edsite_references/gencode.v47.homopolymer_run.bed', sep='\t', index=None, header=None)

  0%|          | 0/33206 [00:00<?, ?it/s]